In [69]:
import fused
import json
import os
import time
from pathlib import Path

In [70]:
# We still need your local paths
PATH_TO_CLAUDE_CONFIG = (
    f"{str(Path.home())}\AppData\Roaming\Claude/claude_desktop_config.json"
)


if not os.path.exists(PATH_TO_CLAUDE_CONFIG):
    # Creating the config file
    os.makedirs(os.path.dirname(PATH_TO_CLAUDE_CONFIG), exist_ok=True)
    with open(PATH_TO_CLAUDE_CONFIG, "w") as f:
        json.dump({}, f)

assert os.path.exists(PATH_TO_CLAUDE_CONFIG), (
    "Please update the PATH_TO_CLAUDE_CONFIG variable with the correct path to your Claude config file"
)

In [71]:
# Local path to the Claude app
CLAUDE_APP_PATH = r"C:\Users\user\AppData\Local\AnthropicClaude\claude.exe"
assert os.path.exists(CLAUDE_APP_PATH), (
    "Please update the CLAUDE_APP_PATH variable with the correct path to your Claude app"
)

In [72]:
# Change this path if you're not running this from the repo root
WORKING_DIR = os.getcwd()

In [73]:
# We'll load the commons folder once again to have our helper functions
commit = "5dda36c"
common = fused.load(
    f"https://github.com/fusedio/udfs/tree/{commit}/public/common"
).utils

module_name='utils' source_code='# To use these functions, add the following command in your UDF:\n# `common = fused.utils.common`\nfrom __future__ import annotations\nimport fused\nimport pandas as pd\nimport numpy as np\nimport json\nimport os\nfrom numpy.typing import NDArray\nfrom typing import Dict, List, Literal, Optional, Sequence, Tuple, Union, Any\nfrom loguru import logger\nfrom fused.api.api import AnyBaseUdf\n\ndef to_pickle(obj):\n    """Encode an object to a pickle byte stream and store in DataFrame."""\n    import pickle\n    import pandas as pd\n    return pd.DataFrame(\n        {"data_type": [type(obj).__name__], "data_content": [pickle.dumps(obj)]}\n    )\n\ndef from_pickle(df):\n    """Decode an object from a DataFrame containing pickle byte stream."""\n    import pickle\n    return pickle.loads(df["data_content"].iloc[0])\n\ndef df_summary(df, description="", n_head=5, n_tail=5, n_sample=5, n_unique=100, add_details=True):\n    val = description+"\\n\\n"\n    val +=

In [74]:
# And see which agents we have available
json.load(open(os.path.join(WORKING_DIR, "agents.json")))

{'agents': [{'name': 'get_current_time', 'udfs': ['current_utc_time']},
  {'name': 'fused_docs', 'udfs': ['list_public_udfs', 'reading_fused_docs']},
  {'name': 'vancouver_open_data',
   'udfs': ['hundred_parks_in_vancouver', 'internet_speeds_for_lat_lon']},
  {'name': 'elevation_stats_for_lat_lon_area', 'udfs': ['elevation_stats']},
  {'name': 'dynamic_output_vector', 'udfs': ['dynamic_output_vector_udf']},
  {'name': 'vancouver_open_data_demo',
   'udfs': ['hundred_parks_in_vancouver',
    'electric_vehicle_chargers_in_vancouver',
    'yearly_crime_amount',
    'internet_speeds_for_lat_lon',
    'community_gardens_vancouver']},
  {'name': 'world_countries_info', 'udfs': ['get_country_data']},
  {'name': 'routing_agent', 'udfs': ['single_routing_agent']},
  {'name': 'stock_information', 'udfs': ['get_stock_details']},
  {'name': 'tourism_info', 'udfs': ['visitors_arrival', 'visitors_purpose']}]}

In [75]:
AGENT_NAME = "tourism_info"

In [76]:
@fused.udf
def visitor_info(year: int = 2024, country: str = None):
    """
    UDF to retrieve visitor arrival information for a given year and optional country.
    Defaults to 2024 if no year is provided. If no country is specified, returns data for all countries.
    """
    import pandas as pd
    import fused
    from io import BytesIO
    import requests
    
    drive_url = "https://drive.google.com/uc?export=download&id=1QoookSZMXWs3eNQ-q0Bsxq4tCB_KH4ux"
    
    # @fused.cache
    def load_csv():
        response = requests.get(drive_url)
        response.raise_for_status()
        return pd.read_csv(BytesIO(response.content))
    
    # Load data
    df = load_csv()
    
    # Convert 'Year' column to integer type
    df['Year'] = df['Year'].astype(int)
    
    # Apply year filter
    result = df[df['Year'] == year]
    
    # Apply country filter if provided, otherwise return all countries
    if country:
        result = result[result['Country'] == country]
    
    # Return the filtered DataFrame or an empty DataFrame if no data is found
    return result if not result.empty else pd.DataFrame(columns=df.columns)



In [77]:
fused.run(visitor_info)

,Year,Growth Rate(%),Country,Visitor Arrivals
60,2024,47.09021457,Japan,"36,870,148"
66,2024,18.8,India,"9,500,000"


In [78]:
visitor_info_metadata = {
    "description": """
Name: visitor_info
Purpose and Functionality:
The UDF 'visitor_info' retrieves visitor arrival data for a specified year from a CSV file. If no year is provided, it defaults to 1964. The function reads the dataset, filters the data based on the requested year, and returns a DataFrame containing visitor statistics.

Input Parameters:
- year (int, optional): The year for which visitor arrival data is requested. Defaults to 1964.

Output:
The function returns a Pandas DataFrame containing visitor arrival details for the specified year. If no records exist for the given year, an empty DataFrame with the appropriate column structure is returned.

Technical Details and Limitations:
- The function reads data from a local CSV file ('visitors_arrival.csv'). The CSV file must be available in the expected format.
- If the CSV file is missing or corrupt, the function may raise an error.
- The function ensures that the 'Year' column is treated as an integer for filtering.
- If an invalid year (e.g., a non-integer string) is provided, the function may fail unless additional validation is implemented.
- The function caches the CSV file using fused.cache to improve efficiency.
    """,
    "parameters": [
        {
            "name": "year",
            "type": "integer",
            "default": 1964,
            "description": "The year for which visitor arrival data is requested. Defaults to 1964."
        }
    ]
}

visitor_info_metadata

{'description': "\nName: visitor_info\nPurpose and Functionality:\nThe UDF 'visitor_info' retrieves visitor arrival data for a specified year from a CSV file. If no year is provided, it defaults to 1964. The function reads the dataset, filters the data based on the requested year, and returns a DataFrame containing visitor statistics.\n\nInput Parameters:\n- year (int, optional): The year for which visitor arrival data is requested. Defaults to 1964.\n\nOutput:\nThe function returns a Pandas DataFrame containing visitor arrival details for the specified year. If no records exist for the given year, an empty DataFrame with the appropriate column structure is returned.\n\nTechnical Details and Limitations:\n- The function reads data from a local CSV file ('visitors_arrival.csv'). The CSV file must be available in the expected format.\n- If the CSV file is missing or corrupt, the function may raise an error.\n- The function ensures that the 'Year' column is treated as an integer for filte

In [79]:
common.save_to_agent(
    agent_json_path=os.path.join(WORKING_DIR, "agents.json"),
    udf=visitor_info,
    agent_name=AGENT_NAME,
    udf_name="visitors_arrival",
    mcp_metadata=visitor_info_metadata,
)

In [80]:
@fused.udf
def get_visitor_data(year: int = None, country: str = None):
    """
    UDF to retrieve visitor arrival data with optional filters.

    Parameters:
    - year (int): Filter by specific year (e.g., 2023)
    - country (str): Filter by specific country name (e.g., 'Thailand')

    Returns:
    A DataFrame with the following columns:
    - Country
    - Purpose_of_visit_to_Japan
    - Year
    - Growth Rate(%)
    - Visitor Arrivals
    """
    import pandas as pd
    import fused
    from io import BytesIO
    import requests

    drive_url = "https://drive.google.com/uc?export=download&id=1OQkx2fLA6oQ3fzNL-aiCFqDr65bp-nAL"

    # @fused.cache
    def load_csv():
        response = requests.get(drive_url)
        response.raise_for_status()
        return pd.read_csv(BytesIO(response.content), encoding='utf-8')

    # Load data
    df = load_csv()

    # Rename for consistency
    df.rename(columns={"Country/Area": "Country"}, inplace=True)

    # Ensure Year is numeric
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')

    # Optional filters
    if year is not None:
        df = df[df['Year'] == year]

    if country:
        df = df[df['Country'] == country]

    # Columns to return
    columns = [
        'Country',
        'Purpose_of_visit_to_Japan',
        'Year',
        'Growth Rate(%)',
        'Visitor Arrivals'
    ]

    return df[columns] if not df.empty else pd.DataFrame(columns=columns)


In [81]:
fused.run(get_visitor_data)

,Country,Purpose_of_visit_to_Japan,Year,Growth Rate(%),Visitor Arrivals
0,China,Tourism,1990,NaN,NaN
1,China,Tourism,1991,NaN,NaN
2,China,Tourism,1992,NaN,"29,147"
3,China,Tourism,1993,-9.239372834,"26,454"
4,China,Tourism,1994,-7.197399259,"24,550"
...,...,...,...,...,...
1045,United States,Business,2020,-86.664479278,"28,857"
1046,United States,Business,2021,-95.515819385,"1,294"
1047,United States,Business,2022,"4,408.191653787","58,336"
1048,United States,Business,2023,104.588933077,"119,349"


In [82]:
get_visitor_data_metadata = {
    "description": """
Name: get_visitor_data
Purpose and Functionality:
The UDF 'get_visitor_data' provides detailed visitor arrival statistics from a structured dataset sourced from a CSV file hosted on Google Drive. It allows optional filtering by year and country, and returns comprehensive information including purpose of visit, growth rate, and number of visitor arrivals.

Input Parameters:
- year (int, optional): The year for which detailed visitor data is requested. If not specified, data for all available years will be returned.
- country (str, optional): The country for which visitor data is requested. If not specified, data for all countries will be returned.

Output:
Returns a Pandas DataFrame with the following columns:
- Country/Area
- Purpose_of_visit_to_Japan
- Year
- Growth Rate(%)
- Visitor Arrivals

If no matching records are found, an empty DataFrame with the above column structure is returned.

Technical Details and Limitations:
- The CSV is fetched from a public Google Drive link using HTTP requests. Internet access is required.
- The 'Year' column is cast to integer for filtering.
- The function gracefully handles missing or empty fields, such as blank growth rates or visitor counts.
- If the source format of the CSV changes, the behavior may be affected.
    """,
    "parameters": [
        {
            "name": "year",
            "type": "integer",
            "default": None,
            "description": "The year to filter visitor data. Optional."
        },
        {
            "name": "country",
            "type": "string",
            "default": None,
            "description": "The country to filter visitor data. Optional."
        }
    ]
}
get_visitor_data_metadata

{'description': "\nName: get_visitor_data\nPurpose and Functionality:\nThe UDF 'get_visitor_data' provides detailed visitor arrival statistics from a structured dataset sourced from a CSV file hosted on Google Drive. It allows optional filtering by year and country, and returns comprehensive information including purpose of visit, growth rate, and number of visitor arrivals.\n\nInput Parameters:\n- year (int, optional): The year for which detailed visitor data is requested. If not specified, data for all available years will be returned.\n- country (str, optional): The country for which visitor data is requested. If not specified, data for all countries will be returned.\n\nOutput:\nReturns a Pandas DataFrame with the following columns:\n- Country/Area\n- Purpose_of_visit_to_Japan\n- Year\n- Growth Rate(%)\n- Visitor Arrivals\n\nIf no matching records are found, an empty DataFrame with the above column structure is returned.\n\nTechnical Details and Limitations:\n- The CSV is fetched f

In [83]:
common.save_to_agent(
    agent_json_path=os.path.join(WORKING_DIR, "agents.json"),
    udf=get_visitor_data,
    agent_name=AGENT_NAME,
    udf_name="visitors_purpose",
    mcp_metadata=get_visitor_data_metadata,
)

In [84]:
# Let's make sure we created our agent properly, with all our UDFs
agents = json.load(open(os.path.join(WORKING_DIR, "agents.json")))
print(json.dumps(agents, indent=4, sort_keys=True))

{
    "agents": [
        {
            "name": "get_current_time",
            "udfs": [
                "current_utc_time"
            ]
        },
        {
            "name": "fused_docs",
            "udfs": [
                "list_public_udfs",
                "reading_fused_docs"
            ]
        },
        {
            "name": "vancouver_open_data",
            "udfs": [
                "hundred_parks_in_vancouver",
                "internet_speeds_for_lat_lon"
            ]
        },
        {
            "name": "elevation_stats_for_lat_lon_area",
            "udfs": [
                "elevation_stats"
            ]
        },
        {
            "name": "dynamic_output_vector",
            "udfs": [
                "dynamic_output_vector_udf"
            ]
        },
        {
            "name": "vancouver_open_data_demo",
            "udfs": [
                "hundred_parks_in_vancouver",
                "electric_vehicle_chargers_in_vancouver",
                "

In [85]:
AGENT_NAME

'tourism_info'

In [86]:
# Finally, we can select which Agent we want to pass to Claude in our MCP server config
common.generate_local_mcp_config(
    config_path=PATH_TO_CLAUDE_CONFIG,
    agents_list=[AGENT_NAME],
    repo_path=WORKING_DIR,
)

In [87]:
# Let's read this Claude Desktop config to see what we're passing
claude_config = json.load(open(PATH_TO_CLAUDE_CONFIG))
print(json.dumps(claude_config, indent=4, sort_keys=True))

{
    "mcpServers": {
        "tourism_info": {
            "args": [
                "run",
                "--directory",
                "C:\\Users\\user\\fused-mcp",
                "main.py",
                "--runtime=local",
                "--udf-names=visitors_arrival,visitors_purpose",
                "--name=tourism_info"
            ],
            "command": "uv"
        }
    }
}
